<a href="https://colab.research.google.com/github/mkirby1995/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/Assignments/Backprop_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [0]:
import numpy as np

X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [0,1,0],
    [1,0,0],
    [1,1,1],
    [0,0,0]
])

y = np.array([[0],[1],[1],[1],[1],[0],[0]])

In [0]:
class NeuralNetwork: 
    def __init__(self, inputs, hidden_nodes, output_nodes):
        # Set up Archietecture 
        self.inputs = inputs
        self.hiddenNodes = hidden_nodes
        self.outputNodes = output_nodes
        
        #Initial weights
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes) #2x3
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes) #3x1
    
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
      
    def ReLU(self, x):
        return x * (x > 0)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        
        #Weighted sume of inputs and hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        #Acivations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        #Final activation of output
        self.activated_output = self.ReLU(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        self.o_error = y - o #error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid to error
        
        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [0]:
model = NeuralNetwork(inputs = 3, hidden_nodes = 2, output_nodes = 1)

model.train(X, y)

In [0]:
for i in range(1000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 50 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(model.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - model.feed_forward(X)))))
    model.train(X,y)

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Import data

In [0]:
from sklearn.datasets import load_digits
digits = load_digits()

In [0]:
X = digits.data
y = digits.target.reshape(-1,1)

In [279]:
df = pd.DataFrame(y, columns=['digit'])
cols = list(range(10))

for col in cols:
   df[str(col)] = (df['digit'] == col).astype(int)
df = df.drop(columns=['digit'])

df.head()

,0,1,2,3,4,5,6,7,8,9
0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0


In [280]:
df['1'].value_counts(normalize=True)

0    0.89872
1    0.10128
Name: 1, dtype: float64

In [0]:
y = np.array(df['1']).reshape(-1,1)

In [0]:
model = NeuralNetwork(inputs = 64, hidden_nodes = 64, output_nodes = 10)

model.train(X, y)

In [283]:
for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 50 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(model.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - model.feed_forward(X)))))
    model.train(X,y)

+---------EPOCH 1---------+
Input: 
 [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
Actual Output: 
 [[0]
 [1]
 [0]
 ...
 [0]
 [0]
 [0]]
Predicted Output: 
 [[3.22793906e+08]
 [3.22793906e+08]
 [3.22793906e+08]
 ...
 [3.22793906e+08]
 [3.22793906e+08]
 [3.22793906e+08]]
Loss: 
 1.041959059937272e+17
+---------EPOCH 2---------+
Input: 
 [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
Actual Output: 
 [[0]
 [1]
 [0]
 ...
 [0]
 [0]
 [0]]
Predicted Output: 
 [[3.80771652e+30]
 [3.80771652e+30]
 [3.80771652e+30]
 ...
 [3.80771652e+30]
 [3.80771652e+30]
 [3.80771652e+30]]
Loss: 
 1.4498705106889215e+61
+---------EPOCH 3---------+
Input: 
 [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater


+---------EPOCH 50---------+
Input: 
 [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
Actual Output: 
 [[0]
 [1]
 [0]
 ...
 [0]
 [0]
 [0]]
Predicted Output: 
 [[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]
Loss: 
 nan
+---------EPOCH 100---------+
Input: 
 [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
Actual Output: 
 [[0]
 [1]
 [0]
 ...
 [0]
 [0]
 [0]]
Predicted Output: 
 [[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]
Loss: 
 nan
+---------EPOCH 150---------+
Input: 
 [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
Actual Output: 
 [[0]
 [1]
 [0]
 ...
 [0]


In [284]:
predictions = model.feed_forward(X)

for i in predictions:
  if i[0] <= 0.5:
    i[0] = 0
  else:
    i[0] = 1

pd.DataFrame(predictions).head(40)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater


,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0


In [285]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions, y)

0.10127991096271564



---



### Create classifier object

In [0]:
class Digit_classifier():
  def __init__(self, inputs = 64, hidden_nodes = 64, output_nodes = 10):
    # Network Archietecture
    self.inputs = inputs
    self.hidden_nodes = hidden_nodes
    self.output_nodes = output_nodes
    
    # Weight vectors
    weights_1 = np.random.randn(self.inputs, self.hidden_nodes)
    weights_2 = np.random.randn(self.hidden_nodes, self.output_nodes)
    
  def sigmoid(self, s):
    return 1 / (1+np.exp(-s))

  def sigmoid_prime(self, s):
    return s * (1 - s)  
  
  def feed_forward(self, X):
    pass
    # Weighted sum of inputs and hidden layer
    # Activations of weighted sum
    # Weighted sum between hidden layer and output layer
    # Activation of output layer
  
  def backpropagation(self, X, y, output):
    pass
  
  def train(self, X, y):
    pass



---



### Piece by piece

In [110]:
weight_1 = np.random.randn(64, 64)
weight_1[0]

array([-0.06957314,  0.10824847,  0.90501819,  0.35491638,  0.82431949,
        0.80888268,  0.44990352,  0.37154659,  0.26191447, -0.34803414,
       -0.42258186,  0.73029588, -0.7914443 ,  1.16446827, -0.14588225,
        0.87926222, -0.17985898, -0.42148567, -1.11251418,  0.6618503 ,
        0.24918437,  1.68579464, -1.48940602,  0.7528745 ,  1.70021297,
       -0.64061628,  0.34436021,  0.45214506, -0.07099602, -0.21061056,
        0.55031029,  0.79202208,  1.05504505,  1.65310985, -0.20516906,
        0.70301353,  0.06351653,  0.29434747,  0.30932039,  0.57938267,
       -1.04344736,  0.67695541, -1.98591741, -1.31515216,  0.87184354,
        1.05159814, -0.87008891, -0.44042582,  0.09900722, -1.55848953,
       -0.36545941, -0.46152401, -0.84879621, -0.22886593,  0.28765513,
       -1.14967063, -1.11063223,  0.34863978, -0.55411973, -0.18367843,
       -0.89833473,  0.95610061,  0.06831715, -1.42508258])

In [112]:
hidden_sum = np.dot(X, weight_1)
hidden_sum

array([[ 28.78577553,  39.52596328, -26.77278355, ...,  14.11408919,
        168.08854758, -18.67872106],
       [ -2.06773189,  38.33974168,  -1.52849426, ..., -11.07581255,
        182.34455544,   6.50167291],
       [ -8.11298215, -17.64653611, 112.20114891, ...,  21.00951745,
        141.51333797,  -0.8440102 ],
       ...,
       [ 35.34617683,  31.54591685,  37.41217027, ...,  23.43606658,
        237.50482205,  20.98327264],
       [ 78.14159301,  93.95257146, -43.74745067, ...,  23.82764406,
        184.43179304, -12.49563087],
       [ 51.72365521,  61.14860388,  42.14489244, ...,  21.50532263,
        210.9279007 ,  22.98997898]])

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?